# Part 0: Mining the web

Perhaps the richest source of openly available data today is [the Web](http://www.computerhistory.org/revolution/networking/19/314)! In this lab, you'll explore some of the basic programming tools you need to scrape web data.

> **Note.** The Vocareum platform runs in a cloud-based environment that limits what websites a program can connect to directly. Therefore, some (or possibly all) of the code below will **not** work. Therefore, we are making this notebook **optional** and are providing solutions inline.
>
> Even if you are using a home or local installation of Jupyter, you may encounter problems if you attempt to access a site too many times or too rapidly. That can happen if your internet service provider (ISP) or the target website detect your accesses as "unusual" and reject them. It's easy to imagine accidentally writing an infinite loop that tries to access a page and being seen from the other side as a malicious program. :)

## The Requests module

Python's [Requests module](http://requests.readthedocs.io/en/latest/user/quickstart/) to download a web page.

For instance, here is a code fragment to download the [Georgia Tech](http://www.gatech.edu) home page and print the first 250 characters. You might also want to [view the source](http://www.computerhope.com/issues/ch000746.htm) of Georgia Tech's home page to get a nicely formatted view, and compare its output to what you see above.

In [1]:
import requests

response = requests.get('https://www.gatech.edu/')
webpage = response.text  # or response.content for raw bytes

print(webpage[0:250]) # Prints the first hundred characters only

<!DOCTYPE html>
<html lang="en" dir="ltr" 
  xmlns:content="http://purl.org/rss/1.0/modules/content/"
  xmlns:dc="http://purl.org/dc/terms/"
  xmlns:foaf="http://xmlns.com/foaf/0.1/"
  xmlns:og="http://ogp.me/ns#"
  xmlns:rdfs="http://www.w3.org/2000


**Exercise 1.** Given the contents of the GT home page as above, write a function that returns a list of links (URLs) of the "top stories" on the page.

For instance, on Friday, September 9, 2016, here was the front page:

![www.gatech.edu as of Fri Sep 9, 2016](./www.gatech.edu--2016-09-09--annotated-medium.png)

The top stories cycle through in the large image placeholder shown above. We want your function to return the list of URLs behind each of the "Full Story" links, highlighted in red. If no URLs can be found, the function should return an empty list.

In [2]:
import re # Maybe you want to use a regular expression?

def get_gt_top_stories(webpage_text):
    """Given the HTML text for the GT front page, returns a list
    of the URLs of the top stories or an empty list if none are
    found.
    """
    # ANSWER:
    pattern = '''<a class="slide-link" href="(?P<url>[^"]+)"'''
    return re.findall(pattern, webpage_text)

In [3]:
top_stories = get_gt_top_stories(webpage)
print("Links to GT's top stories:", top_stories)

Links to GT's top stories: ['https://www.news.gatech.edu/2018/09/10/lessons-courage-andrew-young', 'http://www.president.gatech.edu/publications-presentations/publications/annual-reports/2018/', 'http://news.gatech.edu/features/imagine-future-which-learning-life']


## A more complex example

Go to [Yelp!](http://www.yelp.com) and look up `ramen` in `Atlanta, GA`. Take note of the URL:

![Yelp! search for ramen in ATL](./yelp-search-example.png)

This URL encodes what is known as an _HTTP "get"_ method (or request). It basically means a URL with two parts: a _command_ followed by one or more _arguments_. In this case, the command is everything up to and including the word `search`; the arguments are the rest, where individual arguments are separated by the `&` or `#`.

> "HTTP" stands for "HyperText Transport Protocol," which is a standardized set of communication protocols that allow _web clients_, like your web browser or your Python program, to communicate with _web servers_.

In this next example, let's see how to build a "get request" with the `requests` module. It's pretty easy!

In [4]:
url_command = 'https://yelp.com/search'
url_args = {'find_desc': "ramen",
            'find_loc': "atlanta, ga"}
response = requests.get (url_command, params=url_args, timeout=60)

print ("==> Downloading from: '%s'" % response.url) # confirm URL
print ("\n==> Excerpt from this URL:\n\n%s\n" % response.text[0:10000])


==> Downloading from: 'https://www.yelp.com/search?find_desc=ramen&find_loc=atlanta%2C+ga'

==> Excerpt from this URL:

<!DOCTYPE HTML>

<!--[if lt IE 7 ]> <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie6 ie ltie9 ltie8 no-js" lang="en"> <![endif]-->
<!--[if IE 7 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie7 ie ltie9 ltie8 no-js" lang="en"> <![endif]-->
<!--[if IE 8 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie8 ie ltie9 no-js" lang="en"> <![endif]-->
<!--[if IE 9 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie9 ie no-js" lang="en"> <![endif]-->
<!--[if (gt IE 9)|!(IE)]><!--> <html xmlns:fb="http://www.facebook.com/2008/fbml" class="no-js" lang="en"> <!--<![endif]-->
    <head>
        <script>
            (function() {
                var main = null;

                var main=function(){window.onerror=function(k,a,c,i,f){var j=(document.getElementsByTagName("html")[0].getAttribute("webdriver")==="true"||naviga

**Exercise 2.** Given a search topic, location, and a rank $k$, return the name of the $k$-th item of a Yelp! search. If there is no $k$-th item, return `None`.

> The demo query above only gives you a website with the top 10 items, meaning you could only use it for $k \leq 10$. Figure out how to modify it to solve the problem when $k > 10$.

In [7]:
def find_yelp_item (topic, location, k):
    """Returns the k-th suggested item from Yelp! in Atlanta for the given topic."""
    # ANSWER:
    import re
    if k < 1: return None
        
    # Download page
    url_command = 'https://yelp.com/search'
    url_args = {'find_desc': topic,
                'find_loc': location,
                'start': k-1
               }

    response = requests.get (url_command, params=url_args)

    if not response: return None
    print('Downloaded from: {}'.format(response.url))
    
    # Look for the line containing the name of the k-th item
    item_pattern = re.compile ('<span class="indexed-biz-name">{}\.[ \n\t]*.*<span >(?P<item_name>.*)</span></a>'.format(k))
    
    item_matches = item_pattern.findall(response.text)

    # Return the first match
    if(len(item_matches) > 0):
        return str(item_matches[0])
    
    # No matches, evidently
    return None

In [8]:
assert find_yelp_item('fried chicken', 'Atlanta, GA', -1) is None # Tests an invalid value for 'k'

> Search queries on Yelp! don't always return the same answers, since the site is always changing! Also, your results might not match a query you do via your web browser (_why not?_). As such, you should manually check your answers.

In [7]:
item = find_yelp_item ('fried chicken', 'Atlanta, GA', 1)
print (item)

# The most likely answer on September 11, 2018:
assert item in ['Gus’s World Famous <span class="highlighted">Fried</span> <span class="highlighted">Chicken</span>',
                'Gus’s World Famous Fried Chicken']                

Downloaded from: https://www.yelp.com/search?find_desc=fried+chicken&find_loc=Atlanta%2C+GA&start=0
Gus’s World Famous Fried Chicken


In [8]:
item = find_yelp_item ('fried chicken', 'Atlanta, GA', 5)
print (item)

# The most likely answer on September 11, 2018:
assert item == 'Buttermilk Kitchen'

Downloaded from: https://www.yelp.com/search?find_desc=fried+chicken&find_loc=Atlanta%2C+GA&start=4
Buttermilk Kitchen


In [9]:
item = find_yelp_item('fried chicken', 'Atlanta, GA', 10)
print(item)

# Most likely correct answer as of September 11, 2018:
assert item == 'Colonnade Restaurant'

Downloaded from: https://www.yelp.com/search?find_desc=fried+chicken&find_loc=Atlanta%2C+GA&start=9
Colonnade Restaurant


One issue with the above exercises is that they treat HTML as a flat string, whereas the document is at least semi-structured. Moreover, web pages are such a common source of data today that you would expect better tools for processing them. Indeed, such tools exist! The next part of this assignment, Part 1, walks you through one such tool. So, head there when you are ready!